In [2]:
import torch
from torchvision import models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

Download labeling from GitHub - https://github.com/tzutalin/labelImg


`!pip install pyqt5`

`!pip install lxml`


Installation guide - https://github.com/heartexlabs/labelImg#installation

Run this command in the terminal

`pyrcc5 -o libs/resources.py resources.qrc`

In [ ]:
import torchvision

# Load pre-trained model with ResNet50 backbone
model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True)

# Set the model in evaluation mode
model.eval()

In [ ]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset

In [2]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset


class YOLODataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.imgs = list(sorted(os.listdir(os.path.join(root_dir, "images"))))
        self.labels = list(sorted(os.listdir(os.path.join(root_dir, "labels"))))

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, "images", self.imgs[idx])
        label_path = os.path.join(self.root_dir, "labels", self.labels[idx])

        img = Image.open(img_path).convert("RGB")

        # Read YOLO label file
        with open(label_path, "r") as file:
            lines = file.read().splitlines()

        labels = []
        for line in lines:
            class_id, x_center, y_center, width, height = map(float, line.split())
            labels.append([class_id, x_center, y_center, width, height])

        labels = torch.tensor(labels)

        if self.transform is not None:
            img, labels = self.transform(img, labels)

        return img, labels

In [3]:
train_dataset = YOLODataset("data/train set")
valid_dataset = YOLODataset("data/validation set")

In [4]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [6]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)